# PROYECTO FINAL: ANÁLISIS DE ARTISTAS EN SPOTIFY Y YOUTUBE

## Ejercicio 1: Limpieza de Datos

Identifica las columnas con valores nulos y sigue estas reglas:

* Si una columna tiene más del 50% de valores nulos, elimínala.
* Para columnas numéricas con valores nulos, reemplázalos por la mediana.
* Para columnas categóricas con valores nulos, reemplázalos por el valor más frecuente.
* Cambia los tipos de datos de las siguientes columnas:
* Convierte "Key" y "Tempo" a tipo entero.
* Convierte "Views", "Likes", "Comments", y "Stream" a tipo flotante.
* Guarda el dataset limpio en una variable llamada data_cleaned.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv("../datasets/Spotify_Youtube.csv", index_col=0)


In [ ]:
# Implementación del Ejercicio 1: Limpieza de Datos

# 1. Identificar las columnas con valores nulos
null_counts = data.isnull().sum()

# Eliminar columnas con más del 50% de valores nulos
columns_to_drop = null_counts[null_counts > (0.5 * len(data))].index
data_cleaned = data.drop(columns=columns_to_drop)

# 2. Reemplazar valores nulos en columnas numéricas con la mediana
numeric_columns = data_cleaned.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_columns:
    if data_cleaned[col].isnull().sum() > 0:
        data_cleaned[col].fillna(data_cleaned[col].median(), inplace=True)

# Reemplazar valores nulos en columnas categóricas con el valor más frecuente
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns
for col in categorical_columns:
    if data_cleaned[col].isnull().sum() > 0:
        data_cleaned[col].fillna(data_cleaned[col].mode()[0], inplace=True)

# 3. Cambiar los tipos de datos de las columnas especificadas
# Convierte "Key" y "Tempo" a tipo entero
if 'Key' in data_cleaned.columns:
    data_cleaned['Key'] = data_cleaned['Key'].astype('int', errors='ignore')

if 'Tempo' in data_cleaned.columns:
    data_cleaned['Tempo'] = data_cleaned['Tempo'].astype('int', errors='ignore')

# Convierte "Views", "Likes", "Comments" y "Stream" a tipo flotante
float_columns = ['Views', 'Likes', 'Comments', 'Stream']
for col in float_columns:
    if col in data_cleaned.columns:
        data_cleaned[col] = data_cleaned[col].astype('float', errors='ignore')

# Mostrar un resumen del dataset limpio
data_cleaned.info()

## Ejercicio 2: Análisis Descriptivo del dataset Limpio

Utilizando data_cleaned, calcula las siguientes estadísticas descriptivas para las columnas "Views", "Likes", "Comments", y "Stream":

* Media, mediana, desviación estándar, mínimo, y máximo.
* Guarda estas estadísticas en un dataFrame llamado descriptive_stats.
* Filtra todas las canciones con más de 500 millones de visualizaciones ("Views") y guarda este subconjunto en top_youtube.

In [ ]:
# Ejercicio 2: Análisis Descriptivo del Dataset Limpio

# Calcular estadísticas descriptivas para las columnas numéricas
descriptive_stats = data_cleaned[['Views', 'Likes', 'Comments', 'Stream']].describe()

# Filtrar canciones con más de 500 millones de visualizaciones
top_youtube = data_cleaned[data_cleaned['Views'] > 500000000]

print(descriptive_stats)


## Ejercicio 3: Creación de Nuevas Columnas y Análisis por Grupo

**Utiliza el dataFrame top_youtube.** 

* Crea una nueva columna llamada "Likes_to_Views" que represente la proporción de "Likes" respecto a "Views", con dos decimales.
* Agrupa las canciones de top_youtube por "Album_type" y calcula:
* Promedio y mediana de "Energy" y "Danceability".
* Total de "Views" y "Stream" por tipo de álbum.
* Guarda el resultado de la agrupación en un nuevo dataFrame llamado album_analysis.

In [ ]:
# Crear la nueva columna "Likes_to_Views" como proporción de Likes respecto a Views
top_youtube['Likes_to_Views'] = (top_youtube['Likes'] / top_youtube['Views']).round(2)

# Agrupar por "Album_type" y calcular estadísticas
album_analysis = top_youtube.groupby('Album_type').agg({
    'Energy': ['mean', 'median'],
    'Danceability': ['mean', 'median'],
    'Views': 'sum',
    'Stream': 'sum'
}).reset_index()

print(album_analysis.head())


## Ejercicio 4: Identificación de Canciones con Baja Proporción de Likes

**Utiliza el dataFrame top_youtube.**

* Filtra todas las canciones cuya proporción "Likes_to_Views" sea menor a 0.01.
* Guarda este subconjunto en un dataFrame llamado low_likes.
* Dentro de low_likes, calcula el número total de canciones por "Album_type" y guarda el resultado en un dataFrame llamado low_likes_summary.

In [ ]:
# Filtrar canciones con proporción "Likes_to_Views" menor a 0.01
low_likes = top_youtube[top_youtube['Likes_to_Views'] < 0.01]

# Calcular el número total de canciones por "Album_type"
low_likes_summary = low_likes.groupby('Album_type').size().reset_index(name='count')

low_likes_summary.head()

## Ejercicio 5: Análisis de Tendencias de Canciones con Baja Proporción de Likes

**Utiliza el dataFrame low_likes.**

Crea un gráfico de líneas que muestre:

* La relación promedio entre "Stream" y "Energy" para las canciones en low_likes.
* En el eje X: "Energy".
* En el eje Y: promedio de "Stream".
* Diferencia los tipos de álbum ("Album_type") con colores en el gráfico.
* Asegúrate de incluir título, leyendas, y etiquetas de ejes.

In [ ]:
# Promedio de "Stream" por "Energy" y "Album_type" en el subconjunto `low_likes`
trend_data = low_likes.groupby(['Album_type', 'Energy']).agg({'Stream': 'mean'}).reset_index()


# Personalizar el gráfico
plt.title('Relación entre Energy y Stream para canciones con baja proporción de Likes', fontsize=14)
plt.xlabel('Energy', fontsize=12)
plt.ylabel('Stream (promedio)', fontsize=12)
plt.legend(title='Album Type')
plt.grid(True)
plt.tight_layout()
sns.lineplot(data=trend_data, x='Energy', y='Stream', hue='Album_type', marker='o')


## Ejercicio 6: Exploración de Correlaciones

* Calcula la matriz de correlación para las columnas "Views", "Likes", "Comments", "Stream", "Energy", y "Danceability" en low_likes, para ello utiliza el método corr() de los dataframes.
* Genera un mapa de calor de esta matriz de correlación.
* Asegúrate de incluir una barra de colores, etiquetas de ejes, y un título descriptivo.

In [ ]:
# Calcular la matriz de correlación para las métricas clave
correlation_matrix = low_likes[['Views', 'Likes', 'Comments', 'Stream', 'Energy', 'Danceability']].corr()

# Crear un mapa de calor

# Personalizar el mapa de calor
plt.title('Mapa de Calor de Correlaciones en Canciones con Baja Proporción de Likes', fontsize=16)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)


## Ejercicio 7: Análisis de Artistas

**Utiliza el dataFrame low_likes.**

* Identifica los tres artistas con el mayor total de "Stream" en este subconjunto.
* Presenta un dataFrame con el nombre del artista y el total correspondiente, ordenados de mayor a menor.
* Genera un gráfico de barras que compare estos totales entre los artistas identificados, en el eje X debe ir la variable stream y en el eje Y la variable Artist, para ello utiliza el método barplot() de la librería seaborn.

In [ ]:
# Identificar los tres artistas con el mayor total de "Stream"
top_artists = low_likes.groupby('Artist')['Stream'].sum().nlargest(3).reset_index()

# Personalizar el gráfico
plt.title('Top 3 Artistas por Stream Total en Canciones con Baja Proporción de Likes', fontsize=14)
plt.xlabel('Stream (Total)', fontsize=12)
plt.ylabel('Artist', fontsize=12)
plt.tight_layout()

# Crear un gráfico de barras para comparar los totales
sns.barplot(data=top_artists, x='Stream', y='Artist', palette='viridis')

# Mostrar los datos del análisis
print(top_artists)